In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import math
import pandas as pd
import os

os.chdir(r'\\chem.ox.ac.uk\Research\Vallance Claire\Zhihao\Data\COS\Simulation\3_fold_simulation\S3_O1_C1\Scan')

# Constants
k = 8.987155e9          # Coulomb constant [N m² C⁻²]
dt = 0.5 * 1e-15              # Time step in seconds
num_steps = 500000       # Number of integration steps

# Masses (in kg)
m1 = 12.0 * 1.67377e-27  # Mass of ion 1
m2 = 16.0 * 1.67377e-27  # Mass of ion 2
m3 = 32.0 * 1.67377e-27  # Mass of ion 3

# Charges (in Coulombs)
Q1 = 1 * 1.602e-19      # Charge of ion 1
Q2 = 1 * 1.602e-19      # Charge of ion 2
Q3 = 3 * 1.602e-19      # Charge of ion 3

In [2]:
def calculate_acceleration(r1, r2, r3, Q1_t, Q2_t, Q3_t):
    """
    Calculate the acceleration of each ion due to Coulomb forces.
    """
    # Pre-calculate difference vectors and their norms
    diff12 = r1 - r2
    diff13 = r1 - r3
    diff21 = -diff12   # r2 - r1
    diff23 = r2 - r3
    diff31 = -diff13   # r3 - r1
    diff32 = -diff23   # r3 - r2

    # Compute norms raised to the third power
    norm12_cubed = np.linalg.norm(diff12)**3
    norm13_cubed = np.linalg.norm(diff13)**3
    norm23_cubed = np.linalg.norm(diff23)**3

    # Calculate accelerations using Coulomb force law
    a1 = k / m1 * (Q1_t * Q2_t * diff12 / norm12_cubed +
                   Q1_t * Q3_t * diff13 / norm13_cubed)
    a2 = k / m2 * (Q2_t * Q1_t * (-diff12) / norm12_cubed +
                   Q2_t * Q3_t * diff23 / norm23_cubed)
    a3 = k / m3 * (Q1_t * Q3_t * (-diff13) / norm13_cubed +
                   Q2_t * Q3_t * (-diff23) / norm23_cubed)
    return a1, a2, a3

def calculate_first_step_r(r, v, a):
    """Calculate the position at the first time step using the Verlet algorithm."""
    return r + v * dt + 0.5 * a * dt**2

def simulate_run(RCO, RCS):
    """
    Simulate the system for given initial geometry (R in Å).
    The Verlet integration loop is executed, and only the final momenta

    """
    factor = 1e-10  # Conversion from Å to m

    # Initial positions (in meters)
    r1 = np.array([0.0, 0.0])*factor
    r2 = np.array([RCO , 0.0])*factor
    r3 = np.array([-RCS , 0.0])*factor

    # Initial velocities (m/s)
    v1 = np.array([0.0, 0.0])
    v2 = np.array([0.0, 0.0])
    v3 = np.array([0.0, 0.0])

    # Initial acceleration and first step positions
    a1, a2, a3 = calculate_acceleration(r1, r2, r3, Q1, Q2, Q3)
    r1_new = calculate_first_step_r(r1, v1, a1)
    r2_new = calculate_first_step_r(r2, v2, a2)
    r3_new = calculate_first_step_r(r3, v3, a3)

    # Initialize positions for Verlet integration
    r1_old, r2_old, r3_old = r1, r2, r3
    r1_now, r2_now, r3_now = r1_new, r2_new, r3_new

    # Verlet integration loop
    for step in range(2, num_steps):
        a1_now, a2_now, a3_now = calculate_acceleration(r1_now, r2_now, r3_now, Q1, Q2, Q3)
        r1_new = 2 * r1_now - r1_old + a1_now * dt**2
        r2_new = 2 * r2_now - r2_old + a2_now * dt**2
        r3_new = 2 * r3_now - r3_old + a3_now * dt**2

        # Update velocities (central difference)
        v1 = 0.5 * (r1_new - r1_old) / dt
        v2 = 0.5 * (r2_new - r2_old) / dt
        v3 = 0.5 * (r3_new - r3_old) / dt

        # Prepare for next iteration
        r1_old, r2_old, r3_old = r1_now, r2_now, r3_now
        r1_now, r2_now, r3_now = r1_new, r2_new, r3_new

    # # Final momenta (in kg·m/s)
    # P1 = m1 * np.linalg.norm(v1)
    # P2 = m2 * np.linalg.norm(v2)
    # P3 = m3 * np.linalg.norm(v3)
    # # Convert momenta to atomic units (1 a.u. = 1.99285e-24 kg·m/s)
    # P1_au = P1 / (1.99285e-24)
    # P2_au = P2 / (1.99285e-24)
    # P3_au = P3 / (1.99285e-24)


    # Calculate momentum components for each ion
    p1_x_au = m1 * v1[0]/ (1.99285e-24)
    p2_x_au = m2 * v2[0]/ (1.99285e-24)
    p3_x_au = m3 * v3[0]/ (1.99285e-24)


    # Print final result for this run
    print(f"Final results for RCO = {RCO} Å, RCS = {RCS} Å:")
    print(f"  C momentum = {p1_x_au:.3e} a.u.")
    print(f"  O momentum = {p2_x_au:.3e} a.u.")
    print(f"  S momentum = {p3_x_au:.3e} a.u.")

    #print(f"  S momentum = {P1_au:.3e} a.u., O1 momentum = {P2_au:.3e} a.u., O2 momentum = {P3_au:.3e} a.u.")
   

    return {
        'R_CO': RCO,
        'R_CS': RCS,
        'momentum_C_x_au': p1_x_au,
        'momentum_O_x_au': p2_x_au,
        'momentum_S_x_au': p3_x_au,
    }

In [3]:
def main_simulation():
    """
    Run the simulation for a range of RCO and RCS values, collect the final results,
    and save them to a CSV file.
    """
    results = []
    for RCO in np.arange(1.1, 2.8, 0.1):
        for RCS in np.arange(1.5, 3.2, 0.1):
            result = simulate_run(RCO, RCS)
            results.append(result)
    
    # Create DataFrame and save results to CSV
    df = pd.DataFrame(results)
    df.to_csv("final_simulation_results.csv", index=False)
    print("OCS_ions_info.csv")
    return df

if __name__ == "__main__":
    df = main_simulation()

Final results for RCO = 1.1 Å, RCS = 1.5 Å:
  C momentum = 6.524e+01 a.u.
  O momentum = 2.602e+02 a.u.
  S momentum = -3.255e+02 a.u.
Final results for RCO = 1.1 Å, RCS = 1.6 Å:
  C momentum = 6.492e+01 a.u.
  O momentum = 2.542e+02 a.u.
  S momentum = -3.191e+02 a.u.
Final results for RCO = 1.1 Å, RCS = 1.7000000000000002 Å:
  C momentum = 6.458e+01 a.u.
  O momentum = 2.487e+02 a.u.
  S momentum = -3.133e+02 a.u.
Final results for RCO = 1.1 Å, RCS = 1.8000000000000003 Å:
  C momentum = 6.425e+01 a.u.
  O momentum = 2.437e+02 a.u.
  S momentum = -3.079e+02 a.u.
Final results for RCO = 1.1 Å, RCS = 1.9000000000000004 Å:
  C momentum = 6.391e+01 a.u.
  O momentum = 2.391e+02 a.u.
  S momentum = -3.030e+02 a.u.
Final results for RCO = 1.1 Å, RCS = 2.0000000000000004 Å:
  C momentum = 6.358e+01 a.u.
  O momentum = 2.348e+02 a.u.
  S momentum = -2.984e+02 a.u.
Final results for RCO = 1.1 Å, RCS = 2.1000000000000005 Å:
  C momentum = 6.326e+01 a.u.
  O momentum = 2.308e+02 a.u.
  S momentu